In [197]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import os
from PIL import Image
import io
from base64 import b64encode, b64decode
import sys
import unicodedata
import pickle

In [198]:
def process_single_product(url, base_df, group_name='sas'):
    r = requests.get(url)
    if r.status_code == 200:
        pass
    elif r.status_code >= 400:
        print(url)
        print(r.status_code)
        return

    soup = BeautifulSoup(r.content, 'lxml')

    id_ = soup.find('li', class_="b-product-data__item b-product-data__item_type_sku").span['title']
    name = soup.find('h1', class_="cs-title cs-title_type_product cs-online-edit").span.text
    price = soup.find('p', class_="b-product-cost__price").text
    price = unicodedata.normalize("NFKD",price)
    description = soup.find('div', class_="b-user-content").text
    product_attrs = soup.find_all('td', class_="b-product-info__cell")
    product_attrs = [i.text.strip() + ' ' if a % 2 == 0  else i.text.strip() + '\n' for a, i in enumerate(product_attrs)]
    product_attrs = "".join(product_attrs).strip()
    try:
        available = soup.find('li', class_="b-product-data__item b-product-data__item_type_available").text
    except AttributeError:
        available = "Нет в наличии"
    try:
        img_url = soup.find('img', class_="cs-product-image__img")['src']
        get_img = requests.get(img_url, stream=True)
        with open(rf"{group_name}\{id_}.jpg", 'wb') as f:
            f.write(get_img.content)
        img_path = [[rf"{group_name}\{id_}.jpg"]]
        img2 = soup.find('img', class_="cs-images__img")
        if img2:
            img_url2 = img2['src']
            get_img2 = requests.get(img_url2, stream=True)
            with open(rf"{group_name}\{id_}_2.jpg", 'wb') as f:
                f.write(get_img2.content)
            img_path[0].append(rf"{group_name}\{id_}_2.jpg")
    except TypeError:
        img_path = ["No_Image)-:"]

    product = {"Название": [name], "Цена": [price], "Описание": [description], "Характеристики": [product_attrs],
               "Код": [id_], "Доступность": available, "Изображения": img_path}
    product = pd.DataFrame(product)
    display(product)
    if base_df is not None:
        return pd.concat([base_df, product], ignore_index=True)
    return product

In [199]:
url = r"https://yama-parts.kz/g8100824-zapchasti-dlya-lodochnyh"
root_url = r"https://yama-parts.kz"
headers = {'User-agent': 'Mozilla/5.0'}

In [200]:
r = requests.get(url)
soup = BeautifulSoup(r.content, 'lxml')


## process


In [204]:
for i in soup.find_all('li', class_="cs-product-groups-gallery__item cs-online-edit"):
    group_df = None
    group_name = i.find('a', class_="cs-product-groups-gallery__title").text
    if os.access(group_name, os.F_OK):
        pass
    else:
        os.mkdir(group_name)
    
    image_url = i.find('a', class_="cs-product-groups-gallery__image-link").img['src']
    image_name = i.find('a', class_="cs-product-groups-gallery__image-link").img['alt']
    get_img = requests.get(image_url, stream=True)
    with open(rf"{group_name}\{image_name}.jpg", 'wb') as f:
        f.write(get_img.content)
    
    products_url = root_url + i.find('a', class_="cs-product-groups-gallery__title")['href']
    get_prod_page = requests.get(products_url)
    product_page_soup = BeautifulSoup(get_prod_page.content, 'lxml')
    n_pages = product_page_soup.find('div', class_='b-pager').div['data-pagination-pages-count']
    product_urls_list = [products_url if n < 1 else products_url + f"/page_{n+1}" for n in range(int(n_pages))]
    for product_url in product_urls_list:
        get_prod_page = requests.get(product_url)
        product_page_soup = BeautifulSoup(get_prod_page.content, 'lxml')
        for single_prod in product_page_soup.find_all('a', class_="cs-product-gallery__image-link"):
            single_prod_link = root_url + single_prod['href']
            group_df = process_single_product(single_prod_link, base_df=group_df, group_name=group_name)
            with open(f"{group_name}\\" + b64encode(product_url.encode()).decode() + ".html", 'wb') as f:  # в голос
                f.write(get_prod_page.content)
    group_df.to_pickle(rf"{group_name}\{group_name}.pkl")
    group_df.to_excel(rf"{group_name}\{group_name}.xlsx") 
#     img = Image.open(io.BytesIO(get_img.content))
#     img.save(rf"{group_name}\{image_name}.jpg")
#     print(root_url + i.find('a', class_="cs-product-groups-gallery__title")['href'])
    

,Название,Цена,Описание,Характеристики,Код,Доступность,Изображения
0,Игольчатый Подшипник Шатуна Tohatsu T 25-30 / ...,6 000 Тг.,Подшипник шатунный для TOHATSU/MERCURY 25-30/4...,Производитель Tohatsu\nСтрана производитель Яп...,345-00042-0,В наличии,[Поршневая группа для лодочных моторов\345-000...


,Название,Цена,Описание,Характеристики,Код,Доступность,Изображения
0,Палец поршневой Tohatsu Т 15 332-00021-1,6 000 Тг.,Поршневой палец Tohatsu 332-00021-0\nМощность ...,Производитель Tohatsu\nСтрана производитель Яп...,332-00021-1,В наличии,[Поршневая группа для лодочных моторов\332-000...


,Название,Цена,Описание,Характеристики,Код,Доступность,Изображения
0,Кольца поршневые STD лодочного мотора Suzuki D...,22 000 Тг.,Кольца поршневые Suzuki DF90-115\nМощность дви...,Производитель Suzuki\nСтрана производитель Япо...,1214086FA0,В наличии,[Поршневая группа для лодочных моторов\1214086...


,Название,Цена,Описание,Характеристики,Код,Доступность,Изображения
0,Кольца поршневые Suzuki DT 40 12140-94400-0000,15 000 Тг.,Кольца поршневые для подвесных лодочных моторо...,Производитель Suzuki\nСтрана производитель Япо...,12140-94400-0000,В наличии,[Поршневая группа для лодочных моторов\12140-9...


KeyboardInterrupt: 

In [202]:
group_df

,Название,Цена,Описание,Характеристики,Код,Доступность,Изображения
0,Игольчатый Подшипник Шатуна Tohatsu T 25-30 / ...,6 000 Тг.,Подшипник шатунный для TOHATSU/MERCURY 25-30/4...,Производитель Tohatsu\nСтрана производитель Яп...,345-00042-0,В наличии,[Поршневая группа для лодочных моторов\345-000...
1,Палец поршневой Tohatsu Т 15 332-00021-1,6 000 Тг.,Поршневой палец Tohatsu 332-00021-0\nМощность ...,Производитель Tohatsu\nСтрана производитель Яп...,332-00021-1,В наличии,[Поршневая группа для лодочных моторов\332-000...
2,Кольца поршневые STD лодочного мотора Suzuki D...,22 000 Тг.,Кольца поршневые Suzuki DF90-115\nМощность дви...,Производитель Suzuki\nСтрана производитель Япо...,1214086FA0,В наличии,[Поршневая группа для лодочных моторов\1214086...
3,Кольца поршневые Suzuki DT 40 12140-94400-0000,15 000 Тг.,Кольца поршневые для подвесных лодочных моторо...,Производитель Suzuki\nСтрана производитель Япо...,12140-94400-0000,В наличии,[Поршневая группа для лодочных моторов\12140-9...
4,Кольца поршневые STD лодочного мотора Tohatsu ...,8 000 Тг.,Кольца поршневые STD Tohatsu для подвесных лод...,Производитель Tohatsu\nСтрана производитель Яп...,3B2-00011-0-X,В наличии,[Поршневая группа для лодочных моторов\3B2-000...
...,...,...,...,...,...,...,...
165,"Комплект Tohatsu PARAGON T 4-5 (шатун, 2 подши...",17 000 Тг.,Шатунная сборка для TOHATSU/MERCURY 4/5\nШатун...,Производитель Tohatsu\nСтрана производитель Яп...,369-00040-0-KIT,В наличии,[Поршневая группа для лодочных моторов\369-000...
166,Шатун в сборе с подшипниками Tohatsu PARAGON T...,15 000 Тг.,Шатун для TOHATSU/MERCURY 2.5/3.5\n\n309000400\n,Производитель Tohatsu\nСтрана производитель Яп...,309-00040-0-KIT,В наличии,[Поршневая группа для лодочных моторов\309-000...
167,Комплект поршень палец стопора Tohatsu PARAGON...,10 000 Тг.,Поршень для TOHATSU/MERCURY M2.5A/3.5A/3.5B\n\...,Производитель Tohatsu\nСтрана производитель Яп...,309-00001-0,Заканчивается,[Поршневая группа для лодочных моторов\309-000...
168,"Кольца STD Tohatsu PARAGON T 2,5-3,5 302-00011-0",5 000 Тг.,Поршневое кольцо стандартного размера для 2-х ...,Производитель Tohatsu\nСтрана производитель Яп...,302-00011-0,В наличии,[Поршневая группа для лодочных моторов\302-000...


In [203]:
group_df.to_excel(rf"{group_name}\{group_name}.xlsx")